## Coding Exercise #0703

In [1]:
#Uncomment to force use of CPU.
#import os
#os.environ['CUDA_VISIBLE_DEVICES'] = ''

#Suppress TF warnings
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

### 1. Softmax regression (multi-class logistic regression):

In [1]:
# import tensorflow as tf
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

2024-05-12 18:43:38.318315: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-12 18:43:38.949837: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


#### 1.1. Read in the data:

In [4]:
# We will use Iris data.
# 4 explanatory variables.
# 3 classes for the response variable.
data_raw = load_iris()
data_raw.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [5]:
# Print out the description.
# print(data_raw['DESCR'])

In [6]:
X = data_raw['data']
y = data_raw['target']

In [7]:
# Check the shape.
print(X.shape)
print(y.shape)

(150, 4)
(150,)


#### 1.2. Data pre-processing:

In [8]:
# One-Hot-Encoding.
y = np.array(pd.get_dummies(y, drop_first=False))               # drop_frist = False for one-hot-encoding.
y.shape

(150, 3)

In [9]:
# Scaling
X = scale(X)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=3)
n_train_size = y_train.shape[0]

#### 1.3. Do the necessary definitions:

In [10]:
batch_size = 100                                # Size of each (mini) batch.
n_epochs  = 30000                               # Number of epochs.
learn_rate = 0.05

In [2]:
W = tf.Variable(tf.ones([4,3],dtype=tf.float64))                 # Initial value of the weights = 1.
b = tf.Variable(tf.ones([3],dtype=tf.float64))                   # Initial value of the bias = 1.

2024-05-12 18:43:45.729097: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-12 18:43:45.751118: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-12 18:43:45.751171: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-12 18:43:45.753908: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-12 18:43:45.754024: I external/local_xla/xla/stream_executor

In [12]:
#X_ph = tf.placeholder(tf.float32, shape=(None, 4))    # Number of rows not specified. Number of columns = numbmer of X variables = 4.
#y_ph = tf.placeholder(tf.float32, shape=(None,3))     # Number of rows not specified. Number of columns = number of classes of the y variable = 3.

In [13]:
# Model.
def model(x):
    # Not strictly necessary to apply the softmax activation. => in the end we will apply argmax() function to predict the label!
    # return tf.nn.softmax(tf.matmul(X, W) + b)
    return tf.matmul(x, W) + b

In [14]:
# Loss = cross entropy. 
def loss_fn(y_true, y_pred):
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_true, y_pred))

In [15]:
optimizer = tf.optimizers.SGD(learning_rate=learn_rate)

#### 1.4. Training and Testing:

In [19]:
@tf.function
def train_step(X, y):
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = loss_fn(y, y_pred)
    gradients = tape.gradient(loss, [W, b])
    optimizer.apply_gradients(zip(gradients, [W, b]))

# Training.
for i in range(n_epochs):
    idx_rnd = np.random.choice(range(n_train_size), batch_size, replace=False)  # Random sampling w/o replacement for the batch indices.
    batch_X, batch_y = [X_train[idx_rnd, :], y_train[idx_rnd, :]]               # Get a batch.
    train_step(batch_X, batch_y)
    if (i + 1) % 2000 == 0:
        print("Step : {}".format(i + 1))                                        # Print the step number at every multiple of 2000.

# Testing.
correct_predictions = tf.equal(tf.argmax(y_test, axis=1), tf.argmax(model(X_test), axis=1))  # In argmax(), axis=1 means horizontal direction.
accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))                            # Recast the Boolean as float32 first. Then calculate the mean.
accuracy_value = accuracy.numpy()                                                              # Actually run the test with the test data.

Step : 2000
Step : 4000
Step : 6000
Step : 8000
Step : 10000
Step : 12000
Step : 14000
Step : 16000
Step : 18000
Step : 20000
Step : 22000
Step : 24000
Step : 26000
Step : 28000
Step : 30000


Print the testing result.

In [19]:
print("Accuracy = {:5.3f}".format(accuracy_value))

Accuracy = 0.933
